### import local csv files to aws database for cloud computing 

In [1]:
#import libraries
import os 
import pandas as pd
import numpy as np
import psycopg2

In [2]:
#find csv files in the current working directory
def find_csv_files():
    csv_files = []
    for file in os.listdir(os.getcwd()):
        if file.endswith(".csv"):
            csv_files.append(file)
    return csv_files

In [3]:
#configure local directory
def configure_dataset_directory(csv_files, dataset_dir):
    #create directory
    try:#in case folder exists
        mkdir = "mkdir {0}".format(dataset_dir)
        os.system(mkdir)
    except:
        pass
    #move files to directory
    for file in csv_files:
        move_file ="mv '{0}' {1}".format(file, dataset_dir)
        os.system(move_file)
        #print(move_file)
    return

In [4]:
def create_df(dataset_dir, csv_files):
    #path to csv files
    data_path = os.getcwd()+"/"+dataset_dir+"/"
    #loop through files and create dataframes
    df={}
    for file in csv_files:
        try:
            df[file] = pd.read_csv(data_path+file)
        except UnicodeDecodeError:
            df[file] = pd.read_csv(data_path+file, encoding = "ISO-8859-1")
    return df

In [5]:
# clean table name and column names
import string
#remove all punctuation except for underscore
def remove_punctuation(text):
    punctuation_free = "".join(i for i in text if i not in string.punctuation)
    return punctuation_free
def clean_tbl_name(file):
    #remove punctuation
    file = remove_punctuation(file.lower())
    #remove csv
    return '{0}'.format(file.split('csv')[0])
def clean_col_name(dataframe):
    file_df.columns = [remove_punctuation(col_name.lower()) for col_name in file_df.columns]
    pd2sql_replacements = {
    "object": "varchar",
    "float64": "float",
    "int64": "int",
    "datetime64": "timestamp",
    "timedelta64[ns]": "varchar"
    }
    col_str = ",".join("{} {}".format(n, d) for (n, d) in \
                   zip(file_df.columns, file_df.dtypes.replace(pd2sql_replacements)))
    
    return col_str, dataframe.columns
    

In [6]:
#establish connection to database
def connect_to_db(host, dbname, user, password): 
    try: #default port is 5432
        conn = psycopg2.connect("host=%s dbname=%s user=%s password=%s"%(host, dbname, user, password))
    except psycopg2.OperationalError as e:
        raise e
    else:
        print('database successfully connected!')
        return conn


In [7]:
#main
dataset_dir = "datasets"
csv_files = find_csv_files()
configure_dataset_directory(csv_files, dataset_dir)
df = create_df(dataset_dir, csv_files)

In [8]:
#database credentials
host = "database-lab.cmv8m35efjhm.us-east-1.rds.amazonaws.com" 
dbname = "postgres"
user="postgres"
password = "XXXXXXX"
#establish a connection to db
conn = connect_to_db(host, dbname, user, password)
cursor = conn.cursor()

database successfully connected!


In [9]:
#upload csv files to db through loops
for file in csv_files:  
    file_df = df[file]
    #clean table name and column names
    tbl_name = clean_tbl_name(file)    
    col_str, file_df.columns = clean_col_name(file_df)
    #created new table
    cursor.execute("drop table if exists %s;"%(tbl_name))
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    print("table {0} was created successfully!".format(tbl_name))
    #copy table from csv file
    file_df.to_csv("%s.csv" % tbl_name, header = file_df.columns, index=False)
    my_file = open("%s.csv" % tbl_name, encoding="utf-8")
    #copy 
    SQL_STATEMENT = """
        COPY %s from STDIN with
        csv
        header
        delimiter as ','
        """ 
    cursor.copy_expert(sql=SQL_STATEMENT % (tbl_name), file=my_file)
    #grant public access
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    #close connection
    cursor.close()
    print("table {0} imported to db successfully!".format(tbl_name))